# Pre-processing

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
from scanpy import AnnData
from collections import Counter
import pickle

### Adata datasets
The only specification is that the adata.var genes datasets have to be indexed according to the gene symbols.

### Gene sets
Naturally, also the genes in the gene sets have to be identifyed via gene symbol

### pre_processing
This function takes as parameters:
- adata (the adata object with no previous processing)
- gene_sets (the gene sets organized in a dictionary where keys are the names of the lists and values are the lists
- treshold (genes bellow a certain % of cells will be excluded from the calculations. Default is 0.0)
- n_controls (the number of control lists to be generated, default is 50)

In [2]:
def pre_processing(adata, gene_sets=None, treshold=0.0, n_controls=50):
    #set .var and .obs dtype as object
    adata.var.index = adata.var.index.astype('object')
    adata.obs.index = adata.obs.index.astype('object')
    #making them unique
    adata.var_names_make_unique()
    adata.obs_names_make_unique()
    sc.pp.filter_cells(adata, min_genes=0) #we purposefully do not filter damaged cells since we want them on our dataset
    sc.pp.filter_genes(adata, min_cells=3) #filtering genes in less than 3 cells
    adata.var['mt'] = adata.var_names.str.startswith('MT-')  #annotate the group of mitochondrial and ribosomal protein genes as 'mt' and 'ribo
    adata.var['ribo'] = adata.var_names.str.startswith('RP')
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True) #compute metrics such as % of counts belonguing to the mt and ribo groups
    adata.var['pct_cells_by_count'] = (adata.var['n_cells_by_counts']/adata.shape[0])*100
    #annotate the %of cells in which the transcript appears
    
    all_genes_in_adata_not_in_gene_sets = set(adata.var[adata.var['pct_cells_by_count']>treshold].index.values)
    #preparing the control gene set where the control lists will be taken from. A first filter is to take only genes above a certain threshold if needed.
    
    for key in gene_sets.keys():
        #selecting all eligible control genes 
        all_genes_in_adata_not_in_gene_sets = all_genes_in_adata_not_in_gene_sets.difference(set(gene_sets[key]))
        #separating the control gene set. These genes do not belong in any of the other gene sets
    
    #creating control lists and adding them to the gene_sets dictionary
    for i in range(0, n_controls):
        gene_sets['control_{x}'.format(x=i)] = np.random.choice(list(all_genes_in_adata_not_in_gene_sets), 500)
        #500 random genes not present in the lists are taken as control
        adata.uns['control_list_{x}'.format(x=i)] = gene_sets['control_{x}'.format(x=i)]
        
    if gene_sets is None:
        return adata
    else:
        for key in gene_sets.keys():
            #adata indexes have to be the same as gene_sets nomenclature
            adata.var[key] = adata.var_names.isin(gene_sets[key])
            #computing metrics for each gene set
            sc.pp.calculate_qc_metrics(adata, qc_vars=[key], percent_top=None, log1p=False, inplace=True)
    
    col_names = list(adata.obs.columns.values)
    #selecting all the columns with the control lists metrics
    tot_counts_control_col_names = [col for col in col_names if col.startswith('total_counts_control') is True]
    pct_counts_control_col_names = [col for col in col_names if col.startswith('pct_counts_control') is True]
    tmp_tot_df = adata.obs.loc[:,tot_counts_control_col_names]
    tmp_pct_df = adata.obs.loc[:,pct_counts_control_col_names]
    #aggregating all the control metrics into a a unique column with the mean
    agg_tot_df = tmp_tot_df.agg("mean", axis="columns")
    agg_pct_df = tmp_pct_df.agg("mean", axis="columns")
    #adding aggregated metrics to the adata object
    adata.obs = pd.concat([adata.obs, agg_tot_df, agg_pct_df], axis=1)
    adata.obs.rename(columns={0:'total_counts_control_agg', 1:'pct_counts_control_agg'}, inplace=True)
    adata.obs.drop(columns=tot_counts_control_col_names + pct_counts_control_col_names, inplace=True)
    return adata

In [3]:
#loading the data
adata_10x = sc.read_h5ad(r".\data\datasets\adata_10x.pkl")
adata_all = sc.read_h5ad(r".\data\datasets\adata_all.pkl")
adata_met = sc.read_h5ad(r".\data\datasets\adata_met.pkl")
adata_mouse_1 = sc.read_h5ad(r".\data\datasets\adata_mouse_tagged_1.pkl")
adata_mouse_2 = sc.read_h5ad(r".\data\datasets\adata_mouse_tagged_2.pkl")
adata_mouse_3 = sc.read_h5ad(r".\data\datasets\adata_mouse_tagged_3.pkl")

C:\Users\GM049705\Anaconda3\envs\tesi\lib\site-packages\anndata\_core\anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\GM049705\Anaconda3\envs\tesi\lib\site-packages\anndata\_core\anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
C:\Users\GM049705\Anaconda3\envs\tesi\lib\site-packages\anndata\_core\anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
#loading gene_sets dictionary
with open(r".\data\gene_lists\gene_sets.pkl", 'rb') as fp:
    gene_sets = pickle.load(fp)
    print('gene_sets dictionary loaded')

gene_sets dictionary loaded


In [5]:
gene_sets.keys()

dict_keys(['apex_only', 'frac_seq_only', 'lncrna', 'in_common_apex_frac_seq'])

In [6]:
#remove all warnings
import warnings
warnings.filterwarnings("ignore")
# pre-processing
adata_10x_pp = pre_processing(adata_10x, gene_sets=gene_sets)
adata_all_pp = pre_processing(adata_all, gene_sets=gene_sets)
adata_met_pp = pre_processing(adata_met, gene_sets=gene_sets)
adata_mouse_1_pp = pre_processing(adata_mouse_1, gene_sets=gene_sets)
adata_mouse_2_pp = pre_processing(adata_mouse_2, gene_sets=gene_sets)
adata_mouse_3_pp = pre_processing(adata_mouse_3, gene_sets=gene_sets)

In [13]:
#saving pre-processed data
adata_10x_pp.write(r".\outputs\adata_10x_pp.pkl")
adata_all_pp.write(r".\outputs\adata_all_pp.pkl")
adata_met_pp.write(r".\outputs\adata_met_pp.pkl")
adata_mouse_1_pp.write(r".\outputs\adata_mouse_1_pp.pkl")
adata_mouse_2_pp.write(r".\outputs\adata_mouse_2_pp.pkl")
adata_mouse_3_pp.write(r".\outputs\adata_mouse_3_pp.pkl")